# Run calibration 1.4.2

Here we test running the volcanic forcing time series annually with calibration v1.4.2, developed specifically for this volcanic forcing time series.

Run all 1000 volcanic scenarios + zero volcanic forcing = 1001
Run two SSPs
Run 841 ensemble members
Run one scenario, all ensemble members, zero volcanic with stochastic variability on

1001 x 2 x 841 + 841 = 1684523 ensemble members

Note! scenario members are not temperature-adjusted to rebase to say 1850-1900 or 1995-2014. They should be for reporting.

In [ ]:
import os

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import pooch
import xarray as xr
from tqdm.auto import tqdm

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
scenarios = ["ssp119", "ssp585"]

In [ ]:
df_solar = pd.read_csv(
    "../data/solar-forcing/forcing_yearly_1750-2300.csv", index_col=0
)
df_volcanic_hist = pd.read_csv(
    "../data/volcanic-forcing/yearly/forcing_yearly_1750-2014.csv", index_col=0
)

In [ ]:
volcanic_future = np.zeros((86, 1001))
for volcanic_config in range(1, 1001):  # entry zero = zero mean forcing
    df_yearly_scen = pd.read_csv(
        f"../data/volcanic-forcing/yearly/forcing_yearly_scen{volcanic_config}.csv", index_col=0
    )
    volcanic_future[:, volcanic_config] = df_yearly_scen['Global yearly mean TOA ERF (W/m2)'] - df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].loc[1850:2014].values.mean()

In [ ]:
pl.plot(volcanic_future);

In [ ]:
pl.plot(df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'] - df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].loc[1850:2014].values.mean())
pl.plot(np.arange(2015, 2101), volcanic_future);

In [ ]:
df_solar

In [ ]:
df_methane = pd.read_csv(
    "../data/fair2.1-parameters/calibration-1.4.2/CH4_lifetime.csv",
    index_col=0,
)
df_configs = pd.read_csv(
    "../data/fair2.1-parameters/calibration-1.4.2/calibrated_constrained_parameters.csv",
    index_col=0,
)
valid_all = df_configs.index

In [ ]:
da_emissions = xr.load_dataarray(
    "../data/emissions/ssp_emissions_1750-2500.nc"
)

In [ ]:
da = da_emissions.loc[dict(config="unspecified", scenario=["ssp119", "ssp585"])][:351, ...]
fe = da.expand_dims(dim=["config"], axis=(2))

In [ ]:
volcanic_config_names = [f'volc{i}' for i in range(1001)]

In [ ]:
hist = np.zeros((165, 2, 841, 1))
output = np.zeros((87, 2, 841, 1001))

In [ ]:
for iconf, config in tqdm(enumerate(valid_all), total=len(valid_all)):
    solar_forcing = np.zeros(352)
    volcanic_forcing = np.zeros(352)
    solar_forcing = df_solar["solar_erf"].loc[1750:2101].values

    f = FAIR(ch4_method="Thornhill2021")
    f.define_time(1750, 2101, 1)
    f.define_scenarios(scenarios)
    f.define_configs(volcanic_config_names)
    species, properties = read_properties()
    species.remove("Halon-1202")
    species.remove("NOx aviation")
    species.remove("Contrails")
    f.define_species(species, properties)
    f.allocate()

    # solar and volcanic forcing
    f.forcing.loc[
        dict(
            specie='Volcanic',
            timebounds=1750,
        )
    ] = - df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].loc[1850:2014].values.mean()
    f.forcing.loc[
        dict(
            specie='Volcanic',
            timebounds=np.arange(1751, 2016),
        )
    ] = (
        df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].values - 
        df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].loc[1850:2014].values.mean()
    )[:, None, None]
    f.forcing.loc[
        dict(
            specie='Volcanic',
            timebounds=np.arange(2016, 2102),
        )
    ] = volcanic_future[:, None]

    f.emissions = fe.drop_vars("config") * np.ones((1, 1, 1001, 1))

    fill(
        f.forcing,
        solar_forcing[:, None, None] * df_configs.loc[config, "fscale_solar_amplitude"],
        specie="Solar",
    )
    
    # climate response
    fill(
        f.climate_configs["ocean_heat_capacity"],
        df_configs.loc[config, "clim_c1":"clim_c3"].values,
    )
    fill(
        f.climate_configs["ocean_heat_transfer"],
        df_configs.loc[config, "clim_kappa1":"clim_kappa3"].values,
    )  # not massively robust, since relies on kappa1, kappa2, kappa3 being in adjacent cols
    fill(
        f.climate_configs["deep_ocean_efficacy"],
        df_configs.loc[config, "clim_epsilon"],
    )
    fill(
        f.climate_configs["gamma_autocorrelation"],
        df_configs.loc[config, "clim_gamma"],
    )
    fill(f.climate_configs["sigma_eta"], df_configs.loc[config, "clim_sigma_eta"])
    fill(f.climate_configs["sigma_xi"], df_configs.loc[config, "clim_sigma_xi"])
    #fill(f.climate_configs["seed"], df_configs["seed"])
    fill(f.climate_configs["stochastic_run"], False)
    #fill(f.climate_configs["stochastic_run"], True)
    #fill(f.climate_configs["use_seed"], True)
    fill(f.climate_configs["forcing_4co2"], df_configs.loc[config, "clim_F_4xCO2"])
    
    # species level
    f.fill_species_configs()
    
    # carbon cycle
    fill(f.species_configs["iirf_0"], df_configs.loc[config, "cc_r0"], specie="CO2")
    fill(
        f.species_configs["iirf_airborne"],
        df_configs.loc[config, "cc_rA"],
        specie="CO2",
    )
    fill(
        f.species_configs["iirf_uptake"], df_configs.loc[config, "cc_rU"], specie="CO2"
    )
    fill(
        f.species_configs["iirf_temperature"],
        df_configs.loc[config, "cc_rT"],
        specie="CO2",
    )
    
    # aerosol indirect
    fill(f.species_configs["aci_scale"], df_configs.loc[config, "aci_beta"])
    fill(
        f.species_configs["aci_shape"],
        df_configs.loc[config, "aci_shape_so2"],
        specie="Sulfur",
    )
    fill(
        f.species_configs["aci_shape"],
        df_configs.loc[config, "aci_shape_bc"],
        specie="BC",
    )
    fill(
        f.species_configs["aci_shape"],
        df_configs.loc[config, "aci_shape_oc"],
        specie="OC",
    )
    
    # methane lifetime baseline and sensitivity
    fill(
        f.species_configs["unperturbed_lifetime"],
        df_methane.loc["historical_best", "base"],
        specie="CH4",
    )
    fill(
        f.species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "CH4"],
        specie="CH4",
    )
    fill(
        f.species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "N2O"],
        specie="N2O",
    )
    fill(
        f.species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "VOC"],
        specie="VOC",
    )
    fill(
        f.species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "NOx"],
        specie="NOx",
    )
    fill(
        f.species_configs["ch4_lifetime_chemical_sensitivity"],
        df_methane.loc["historical_best", "HC"],
        specie="Equivalent effective stratospheric chlorine",
    )
    fill(
        f.species_configs["lifetime_temperature_sensitivity"],
        df_methane.loc["historical_best", "temp"],
    )
    
    # emissions adjustments for N2O and CH4
    fill(f.species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
    fill(f.species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
    fill(f.species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")
    
    # aerosol direct
    for specie in [
        "BC",
        "CH4",
        "N2O",
        "NH3",
        "NOx",
        "OC",
        "Sulfur",
        "VOC",
        "Equivalent effective stratospheric chlorine",
    ]:
        fill(
            f.species_configs["erfari_radiative_efficiency"],
            df_configs.loc[config, f"ari_{specie}"],
            specie=specie,
        )
    
    # forcing scaling
    for specie in [
        "CO2",
        "CH4",
        "N2O",
        "Stratospheric water vapour",
        "Light absorbing particles on snow and ice",
        "Land use",
    ]:
        fill(
            f.species_configs["forcing_scale"],
            df_configs.loc[config, f"fscale_{specie}"],
            specie=specie,
        )
    
    for specie in [
        "CFC-11",
        "CFC-12",
        "CFC-113",
        "CFC-114",
        "CFC-115",
        "HCFC-22",
        "HCFC-141b",
        "HCFC-142b",
        "CCl4",
        "CHCl3",
        "CH2Cl2",
        "CH3Cl",
        "CH3CCl3",
        "CH3Br",
        "Halon-1211",
        "Halon-1301",
        "Halon-2402",
        "CF4",
        "C2F6",
        "C3F8",
        "c-C4F8",
        "C4F10",
        "C5F12",
        "C6F14",
        "C7F16",
        "C8F18",
        "NF3",
        "SF6",
        "SO2F2",
        "HFC-125",
        "HFC-134a",
        "HFC-143a",
        "HFC-152a",
        "HFC-227ea",
        "HFC-23",
        "HFC-236fa",
        "HFC-245fa",
        "HFC-32",
        "HFC-365mfc",
        "HFC-4310mee",
    ]:
        fill(
            f.species_configs["forcing_scale"],
            df_configs.loc[config, "fscale_minorGHG"],
            specie=specie,
        )
    
    # ozone
    for specie in [
        "CH4",
        "N2O",
        "Equivalent effective stratospheric chlorine",
        "CO",
        "VOC",
        "NOx",
    ]:
        fill(
            f.species_configs["ozone_radiative_efficiency"],
            df_configs.loc[config, f"o3_{specie}"],
            specie=specie,
        )
    
    # tune down volcanic efficacy
    fill(f.species_configs["forcing_efficacy"], 0.6, specie="Volcanic")
    
    # initial condition of CO2 concentration (but not baseline for forcing calculations)
    fill(
        f.species_configs["baseline_concentration"],
        df_configs.loc[config, "cc_co2_concentration_1750"],
        specie="CO2",
    )
    
    # initial conditions
    initialise(f.concentration, f.species_configs["baseline_concentration"])
    initialise(f.forcing, 0)
    initialise(f.temperature, 0)
    initialise(f.cumulative_emissions, 0)
    initialise(f.airborne_emissions, 0)

    f.run(progress=False)
    hist[:, :, iconf, 0] = f.temperature.sel(timebounds=np.arange(1850, 2015), layer=0, config='volc0')
    output[:, :, iconf, :] = f.temperature.sel(timebounds=np.arange(2015, 2102), layer=0)

In [ ]:
# SAVE OUT THE HIST AND OUTPUT
os.makedirs('../output', exist_ok=True)

In [ ]:
ds = xr.Dataset(
    data_vars = dict(
        temperature_historical = (["timebounds_historical", "scenario", "config", "volcanic_historical"], hist),
        temperature_future = (["timebounds_future", "scenario", "config", "volcanic_future"], output),
    ),
    coords = dict(
        timebounds_historical = np.arange(1850, 2015),
        timebounds_future = np.arange(2015, 2102),
        scenario = scenarios,
        config = valid_all,
        volcanic_historical = np.array([0]),
        volcanic_future = np.arange(1001),
    )
)

In [ ]:
# intermediate dump
ds.to_netcdf('../output/stochastic_volcanoes.nc')

In [ ]:
# do stochastic run
solar_forcing = np.zeros(352)
volcanic_forcing = np.zeros(352)
solar_forcing = df_solar["solar_erf"].loc[1750:2101].values

f = FAIR(ch4_method="Thornhill2021")
f.define_time(1750, 2101, 1)
f.define_scenarios(scenarios)
f.define_configs(valid_all)
species, properties = read_properties()
species.remove("Halon-1202")
species.remove("NOx aviation")
species.remove("Contrails")
f.define_species(species, properties)
f.allocate()

# solar and volcanic forcing
f.forcing.loc[
    dict(
        specie='Volcanic',
        timebounds=1750,
    )
] = - df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].loc[1850:2014].values.mean()
f.forcing.loc[
    dict(
        specie='Volcanic',
        timebounds=np.arange(1751, 2016),
    )
] = (
    df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].values - 
    df_volcanic_hist['Global yearly mean TOA ERF (W/m2)'].loc[1850:2014].values.mean()
)[:, None, None]
f.forcing.loc[
    dict(
        specie='Volcanic',
        timebounds=np.arange(2016, 2102),
    )
] = volcanic_future[:, 0, None, None]

f.emissions = fe.drop_vars("config") * np.ones((1, 1, 841, 1))

fill(
    f.forcing,
    solar_forcing[:, None, None] * df_configs.loc[config, "fscale_solar_amplitude"],
    specie="Solar",
)

# climate response
fill(
    f.climate_configs["ocean_heat_capacity"],
    df_configs.loc[:, "clim_c1":"clim_c3"].values,
)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "clim_kappa1":"clim_kappa3"].values,
)  # not massively robust, since relies on kappa1, kappa2, kappa3 being in adjacent cols
fill(
    f.climate_configs["deep_ocean_efficacy"],
    df_configs.loc[:, "clim_epsilon"],
)
fill(
    f.climate_configs["gamma_autocorrelation"],
    df_configs.loc[:, "clim_gamma"],
)
fill(f.climate_configs["sigma_eta"], df_configs.loc[:, "clim_sigma_eta"])
fill(f.climate_configs["sigma_xi"], df_configs.loc[:, "clim_sigma_xi"])
fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], False)
fill(f.climate_configs["stochastic_run"], True)
fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs.loc[:, "clim_F_4xCO2"])

# species level
f.fill_species_configs()

# carbon cycle
fill(f.species_configs["iirf_0"], df_configs.loc[:, "cc_r0"], specie="CO2")
fill(
    f.species_configs["iirf_airborne"],
    df_configs.loc[:, "cc_rA"],
    specie="CO2",
)
fill(
    f.species_configs["iirf_uptake"], df_configs.loc[:, "cc_rU"], specie="CO2"
)
fill(
    f.species_configs["iirf_temperature"],
    df_configs.loc[:, "cc_rT"],
    specie="CO2",
)

# aerosol indirect
fill(f.species_configs["aci_scale"], df_configs.loc[:, "aci_beta"])
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_so2"],
    specie="Sulfur",
)
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_bc"],
    specie="BC",
)
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_oc"],
    specie="OC",
)

# methane lifetime baseline and sensitivity
fill(
    f.species_configs["unperturbed_lifetime"],
    df_methane.loc["historical_best", "base"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "CH4"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "N2O"],
    specie="N2O",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "VOC"],
    specie="VOC",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "NOx"],
    specie="NOx",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane.loc["historical_best", "HC"],
    specie="Equivalent effective stratospheric chlorine",
)
fill(
    f.species_configs["lifetime_temperature_sensitivity"],
    df_methane.loc["historical_best", "temp"],
)

# emissions adjustments for N2O and CH4
fill(f.species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
fill(f.species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
fill(f.species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")

# aerosol direct
for specie in [
    "BC",
    "CH4",
    "N2O",
    "NH3",
    "NOx",
    "OC",
    "Sulfur",
    "VOC",
    "Equivalent effective stratospheric chlorine",
]:
    fill(
        f.species_configs["erfari_radiative_efficiency"],
        df_configs.loc[:, f"ari_{specie}"],
        specie=specie,
    )

# forcing scaling
for specie in [
    "CO2",
    "CH4",
    "N2O",
    "Stratospheric water vapour",
    "Light absorbing particles on snow and ice",
    "Land use",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs.loc[:, f"fscale_{specie}"],
        specie=specie,
    )

for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs.loc[:, "fscale_minorGHG"],
        specie=specie,
    )

# ozone
for specie in [
    "CH4",
    "N2O",
    "Equivalent effective stratospheric chlorine",
    "CO",
    "VOC",
    "NOx",
]:
    fill(
        f.species_configs["ozone_radiative_efficiency"],
        df_configs.loc[:, f"o3_{specie}"],
        specie=specie,
    )

# tune down volcanic efficacy
fill(f.species_configs["forcing_efficacy"], 0.6, specie="Volcanic")

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs["baseline_concentration"],
    df_configs.loc[:, "cc_co2_concentration_1750"],
    specie="CO2",
)

# initial conditions
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

f.run()
stochastic_output = f.temperature.sel(layer=0, timebounds=np.arange(1850, 2102))

In [ ]:
stochastic_output.shape

In [ ]:
pl.plot(np.arange(1850, 2102), stochastic_output[:, 1, :]);

In [ ]:
pl.plot(np.arange(1850, 2015), stochastic_output[:165, 1, :] - hist[:, 1, :, 0]);
pl.plot(np.arange(2015, 2102), stochastic_output[165:, 1, :] - output[:, 1, :, 0]);

In [ ]:
ds_sf = xr.Dataset(
    data_vars = dict(
        temperature = (["timebounds", "scenario", "config", "volcanic"], stochastic_output.data[..., None]),
    ),
    coords = dict(
        timebounds = np.arange(1850, 2102),
        scenario = scenarios,
        config = valid_all,
        volcanic = np.array([0]),
    )
)

In [ ]:
# final dump
ds_sf.to_netcdf('../output/stochastic_volcanoes_stochastic_climate.nc')